# Arbre et Trie (correction)

In [1]:
from pyquickhelper.ipythonhelper import add_notebook_menu
add_notebook_menu()

## Exercice 1


In [2]:
import random
def mot_alea (l) :
    l = [ chr(97+random.randint(0,25)) for i in range(l) ]
    return "".join(l)

taille  = 20
N       = 10000
mots    = [ mot_alea(taille) for _ in range (N) ]
print(len(mots))

10000


## Exercice 2

In [3]:
import time
debut = time.clock()
for k in mots :
    i = mots.index(k)
fin = time.clock()
print ("recherche simple",fin - debut)

recherche simple 1.5386846000127434


Avec ``%timeit`` :

In [4]:
%timeit for k in mots : i = mots.index(k)

1 loops, best of 3: 1.42 s per loop


## Exercice 3 : recherche dichotomique

In [5]:
def dicho (mots, x) :
    a = 0
    b = len(mots)-1
    while a < b :
        m = (a+b)//2
        t = mots[m] 
        if t < x : 
            b = m-1 
        elif t == x :
            return m
        else :
            a = m+1
    return a

mots.sort()

debut = time.clock()
for k in mots :
    i = dicho(mots, k)
fin = time.clock()
print ("dichotomie",fin - debut)

dichotomie 0.11440034518783193


In [6]:
%timeit for k in mots : i = dicho(mots, k)

10 loops, best of 3: 65.8 ms per loop


## Exercice 4

In [8]:
import math

for N in [10, 100, 1000, 10000, 100000] :
    mots    = [ mot_alea(taille) for _ in range (N) ]
    tolook  = [ mots[ random.randint(0,len(mots)-1) ] for i in range(0,1000) ]
    mots.sort()
    
    debut = time.clock()
    for k in tolook :
        i = mots.index(k)
    fin = time.clock()    
    ds = fin-debut
    
    debut = time.clock()
    for k in tolook :
        i = dicho(mots, k)
    fin = time.clock()    
    dd = fin-debut
    
    print(N, "simple",ds, "dicho",dd, "ratio", ds / dd, " ratio théorique ", len(mots)/math.log(len(mots)) * math.log(2)/30)

10 simple 0.0008582659521323421 dicho 0.002135615428443316 ratio 0.40188225871637656  ratio théorique  0.1003433318879937
100 simple 0.002305814655613858 dicho 0.0038175087964873455 ratio 0.6040103058139742  ratio théorique  0.5017166594399687
1000 simple 0.019075997803668088 dicho 0.005086733184072045 ratio 3.7501471206314223  ratio théorique  3.3447777295997914
10000 simple 0.1779432920122872 dicho 0.006568492787494051 ratio 27.090429687473925  ratio théorique  25.085832971998425
100000 simple 5.5287581101214585 dicho 0.009310496417270997 ratio 593.8199063011932  ratio théorique  200.68666377598748


In [12]:
for N in [10, 100, 1000, 10000, 100000] :
    print("N=",N)
    mots    = [ mot_alea(taille) for _ in range (N) ]
    tolook  = [ mots[ random.randint(0,len(mots)-1) ] for i in range(0,1000) ]
    mots.sort()
    
    %timeit for k in tolook : i = mots.index(k)
    %timeit for k in tolook : i = dicho(mots, k)

N= 10
1000 loops, best of 3: 402 µs per loop
1000 loops, best of 3: 1.61 ms per loop
N= 100
100 loops, best of 3: 1.99 ms per loop
100 loops, best of 3: 3.12 ms per loop
N= 1000
100 loops, best of 3: 17.4 ms per loop
100 loops, best of 3: 4.83 ms per loop
N= 10000
10 loops, best of 3: 186 ms per loop
100 loops, best of 3: 6.59 ms per loop
N= 100000
1 loops, best of 3: 5.5 s per loop
100 loops, best of 3: 8.43 ms per loop


Soit $N$ le nombre de mots dans la liste :

* coût de la recherche simple : $O(N)$ 
* coût de la recherche dichotomique : $O(\ln N)$ 

Le ratio $N/\ln N$ mesuré en pratique devrait être sensiblement égal au ratio théorique à un facteur multiplication près. Le trie du tableau qui précède la recherche dichotomique n'est pas pris en compte. Plus on effectue de recherche, plus son coût devient marginal. Ce coût explique néanmoins pourquoi on ne fait pas toujours une recherche dichotomique.

## Exercice 5 : trie

In [8]:
def build_trie(mots) :
    trie = { }
    for m in mots :
        r = trie
        for c in m :
            if c not in r : r[c] = { }
            r = r[c]
    return trie

mots = [ "aaa", "aba", "aab", "baa", "bbb", "bba", "bab" ]

trie = build_trie(mots)
print(trie)

{'a': {'a': {'a': {}, 'b': {}}, 'b': {'a': {}}}, 'b': {'a': {'a': {}, 'b': {}}, 'b': {'a': {}, 'b': {}}}}


Quelques explications supplémentaires concernant cette correction :

* [Question à propos de trie](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/questions/question_2014.html#question-a-propos-de-trie)

## Exercice 6 : recherche dans un trie

In [9]:
def lookup(trie, m) :
    r = trie
    for c in m :
        if c in r :
            r = r[c]
        else :
            return False
    return True

for k in mots :
    print(k, lookup(trie, k))
print("bcc", lookup(trie, "bcc"))    

aaa True
aba True
aab True
baa True
bbb True
bba True
bab True
bcc False


Pour illustrer la structure d'arbre du trie, on l'affiche avec la fonction suivante : 

In [17]:
def print_trie(trie, niveau = 0):
    for k,v in sorted(trie.items()):
        print("  " * niveau + k)
        if len(v) > 0 :
            print_trie(v, niveau+1)
print_trie(trie)

a
  a
    a
    b
  b
    a
b
  a
    a
    b
  b
    a
    b


Il reste un inconvénient à cette représentation. Si on construire le trie pour le mot ``["aaa"]`` ou les mots ``["aa","aaa]"``, on obtient le même trie :

In [18]:
print_trie (build_trie( ["aaa"]) )

a
  a
    a


In [19]:
print_trie (build_trie( ["aaa", "aa"]) )

a
  a
    a


Pour éviter cela, le plus simple est de représenter la fin d'un mot comme un caractère à part entière.

In [23]:
print_trie (build_trie( ["aaa*"]) )

a
  a
    a
      *


In [24]:
print_trie (build_trie( ["aaa*", "aa*"]) )

a
  a
    *
    a
      *


## Exercice 7

Soit $L$ la longueur maximale des mots et $C$ le nombre de lettres distinctes, avec un trie, le coût de la recherche est majoré par : $O(L \ln C)$. On reprend le code de l'exercice 5 et on ajoute le code associé au trie. On effectue 10000 recherches au lieu de 1000 pour avoir une meilleure estimation de la différence (pour vous en convaincre, il suffit comparer les temps obtenus par deux exécution de ce même code).

In [17]:
for N in [10, 100, 1000, 10000, 100000, 200000, 400000] :
    mots    = [ mot_alea(taille) for _ in range (N) ]
    tolook  = [ mots[ random.randint(0,len(mots)-1) ] for i in range(0,10000) ]
    trie    = build_trie(mots)
    mots.sort()
    
    debut = time.clock()
    for k in tolook :
        i = dicho(mots, k)
    fin = time.clock()    
    dd = fin-debut
    
    debut = time.clock()
    for k in tolook :
        i = lookup(trie, k)
    fin = time.clock()
    dt = fin - debut

    print(N, "dicho",dd, "trie", dt)

10 dicho 0.017959867193440004 trie 0.0209845383838001
100 dicho 0.032061172510111646 trie 0.022669425205549487
1000 dicho 0.04955876491817435 trie 0.037136787151666795
10000 dicho 0.06641062658968622 trie 0.05320179812497372
100000 dicho 0.08635643652428371 trie 0.05857119886968576
200000 dicho 0.09092786802807495 trie 0.0707237657989026
400000 dicho 0.11014284761267845 trie 0.0812094066291138


In [18]:
for N in [10, 100, 1000, 10000, 100000, 200000, 400000] :
    print("N=",N)
    mots    = [ mot_alea(taille) for _ in range (N) ]
    tolook  = [ mots[ random.randint(0,len(mots)-1) ] for i in range(0,10000) ]
    trie    = build_trie(mots)
    mots.sort()
    
    %timeit for k in tolook : i = dicho(mots, k)
    %timeit for k in tolook : i = lookup(trie, k)

N= 10
10 loops, best of 3: 16.8 ms per loop
10 loops, best of 3: 26.6 ms per loop
N= 100
10 loops, best of 3: 35.3 ms per loop
10 loops, best of 3: 22.3 ms per loop
N= 1000
10 loops, best of 3: 52.3 ms per loop
10 loops, best of 3: 52.3 ms per loop
N= 10000
10 loops, best of 3: 69.1 ms per loop
10 loops, best of 3: 52.4 ms per loop
N= 100000
10 loops, best of 3: 84.3 ms per loop
10 loops, best of 3: 56.7 ms per loop
N= 200000
10 loops, best of 3: 89.4 ms per loop
10 loops, best of 3: 57 ms per loop
N= 400000
10 loops, best of 3: 121 ms per loop
10 loops, best of 3: 59 ms per loop


Encore une fois, le temps de construction du trie n'est pas pris en compte. Plus il y a de recherche à effectuer, plus il devient négligeable.

Le dictionnaire est un object courant dans la plupart des languages. En python, celui-ci utilise une [table de hachage](http://fr.wikipedia.org/wiki/Table_de_hachage) et le coût d'accès à un élément n'est pas en $O(\ln n)$ mais en $O(n)$ (voir [time complexity](https://wiki.python.org/moin/TimeComplexity)). En C++, le dictionnaire (ou [map](http://www.cplusplus.com/reference/map/map/)) utilise un arbre binaire et l'accès à un élément a un coût logarithmique : [Standard C++ Containers](http://www.cs.northwestern.edu/~riesbeck/programming/c++/stl-summary.html).

## Plus en détails

La recherche dichotomique est équivalente à celle opérée avec un [arbre binaire de recherche](http://fr.wikipedia.org/wiki/Arbre_binaire_de_recherche) (si ce dernier est [équilibré](http://fr.wikipedia.org/wiki/Arbre_B) ou [arbre rouge/noir](http://fr.wikipedia.org/wiki/Arbre_bicolore)). Ce dernier consiste à classer les éléments par ordre alphabétique. Un arbre est souvent représenté par une classe et non par un dictionnaire comme la dernière partie de cette session le laissait supposer.

In [5]:
class Arbre:
    def __init__(self, value):
        self.value = value
        self.children = [ ]
        
    def add_child(self, child):
        self.children.append(child)
        
    def __str__(self):
        rows = [ "value={0}".format(self.value) ]
        for c in self.children:
            s = str(c)
            lines = [ "    " + l for l in s.split("\n") ]
            rows.extend(lines)
        return "\n".join(rows)
    
root = Arbre("racine")
child1 = Arbre("child 1")
child1.add_child ( Arbre("child 2") )
child1.add_child ( Arbre("child 1000") )
root.add_child(child1)
root.add_child( Arbre ("child 3") )
print(root)     

value=racine
    value=child 1
        value=child 2
        value=child 1000
    value=child 3


Les arbres sont des graphes particuliers car il ne contiennent pas de cycles. Il est possible de parcourir les noeuds, de les numéroter. Ils sont très utilisés en machine learning avec les [arbres de décisions](http://fr.wikipedia.org/wiki/Arbre_de_d%C3%A9cision) ou les [random forests](http://en.wikipedia.org/wiki/Random_forest). Ils sont parfois cachés comme dans le cas de la recherche dichotomique qui peut être implémentée à partir d'une structure d'arbre.

Dans le cas de le recherche dichotomique, on suppose que le nombre de noeuds fils est toujours 2. L'ordre alphabétique est le suivant : noeuds fils 1, noeud courant, noeud fils 2. Les deux noeuds fils pourraient être nuls. L'implémentation de l'arbre serait la suivante :

In [9]:
class ArbreDicho:
    def __init__(self, value):
        self.value = value
        self.before = None
        self.after = None
        
    def __str__(self):
        return "value={0}".format(self.value)
        
    def add_before(self, child):
        self.before = child

    def add_after(self, child):
        self.after = child
        
    def find(self, word):
        if self.value == word : return self
        elif word < self.value : 
            if self.before == None : return None
            else : return self.before.find(word)
        else : 
            if self.after == None : return None
            else : return self.after.find(word)
            
    def sorted_list(self):
        res = [ ]
        if self.before != None : res.extend ( self.before.sorted_list() )
        res.append(self.value)
        if self.after != None : res.extend ( self.after.sorted_list() )
        return res

# on crée un arbre dont les noeuds vérifient la propriété énoncé plus haut (les mots apparaissent dans le bon ordre)
root = ArbreDicho("milieu")
root.add_before(ArbreDicho("avant"))
root.add_after(ArbreDicho("zillion"))
root.before.add_before(ArbreDicho("alphabet"))
root.before.add_after(ArbreDicho("avant après"))

# on vérifie que c'est bien le cas
all = root.sorted_list()
assert all == sorted(all)
print(all)

# on effectue la recherche
for a in all:
    f = root.find(a)
    print(f)

['alphabet', 'avant', 'avant après', 'milieu', 'zillion']
value=alphabet
value=avant
value=avant après
value=milieu
value=zillion
